In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from math import sqrt

# De scikit-learn...

# Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PowerTransformer

# Estandarización y modelado
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn import metrics

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_selector
# Division de los datos
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Normalización
from sklearn.preprocessing import MinMaxScaler

# Regresiones
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso, ElasticNet, RidgeCV, ElasticNetCV, LassoCV, LogisticRegression

# ML metricas
from sklearn.metrics import mean_squared_error, r2_score, classification_report, roc_curve, auc, confusion_matrix, accuracy_score, ConfusionMatrixDisplay, mean_absolute_percentage_error, mean_absolute_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Validación Cruzada
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold, LeaveOneOut

# Explicabilidad
import shap

# Redes Neuronales
from setuptools import setup
from distutils.core import setup
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Precision
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam

# Optimización de HP
import optuna
from optuna.trial import Trial


/home/jester/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-21 21:09:46.607902: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-21 21:09:46.610595: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-21 21:09:46.641385: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-21 21:09:47.324766: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('weatherAUS.csv',usecols=range(1,25))

pipeline que limpia datos

In [3]:
class ColDropper(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.drop(["Unnamed: 0", "Date"], axis=1)


class LocDropper(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        aus_loc = [
            " Adelaide",
            "Canberra",
            "Cobar",
            "Dartmoor",
            "Melbourne",
            "MelbourneAirport",
            "MountGambier",
            "Sydney",
            "SydneyAirport",
        ]
        return X[X["Location"].isin(aus_loc)]


class CatFiller(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X["WindGustDir"] = X.groupby("Location")["WindGustDir"].transform(
            lambda x: x.fillna(x.mode()[0])
        )
        X["WindDir9am"] = X.groupby("Location")["WindDir9am"].transform(
            lambda x: x.fillna(x.mode()[0])
        )
        X["WindDir3pm"] = X.groupby("Location")["WindDir3pm"].transform(
            lambda x: x.fillna(x.mode()[0])
        )

        return X


class NumFiller(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        remanining_vnul_columns = X.columns[X.isna().any()].tolist()
        for col in remanining_vnul_columns:
            X[col] = X[col].fillna(X[col].mean())

        return X


# https://www.mdpi.com/2078-2489/13/4/163 Como las variables de la dirección de los vientos pueden tener hasta 16 direcciones diferentes, para convertirlos a variables numéricas, se tiene encuenta una distribución circular. Por eso, cada una de las variables se dividió en dos: Una con el seno y otra con el coseno del angulo
class CoordRecat(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        coord = {
            "N": 0,
            "NNE": 22.5,
            "NE": 45,
            "ENE": 67.5,
            "E": 90,
            "ESE": 112.5,
            "SE": 135,
            "SSE": 157.5,
            "S": 180,
            "SSW": 202.5,
            "SW": 225,
            "WSW": 247.5,
            "W": 270,
            "WNW": 292.5,
            "NW": 315,
            "NNW": 337.5,
        }

        # Aplicar la recategorización
        for col in ["WindGustDir", "WindDir9am", "WindDir3pm"]:
            X[col] = X[col].map(coord)
            X[f"{col}_rad"] = np.deg2rad(X[col])
            X[f"{col}_sin"] = np.sin(X[f"{col}_rad"]).round(5)
            X[f"{col}_cos"] = np.cos(X[f"{col}_rad"]).round(5)

        # Eliminar columnas originales y columnas radianes
        columns_to_drop = [
            f"{col}_rad" for col in ["WindGustDir", "WindDir9am", "WindDir3pm"]
        ] + ["WindGustDir", "WindDir9am", "WindDir3pm"]
        X = X.drop(columns=columns_to_drop, axis=1)

        return X


class LocEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        dummies = pd.get_dummies(X["Location"], dtype=int)
        X = pd.concat([X, dummies], axis=1)
        X.drop("Location", axis=1, inplace=True)

        return X


class ResetIndex(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.reset_index(drop=True)


class Standarizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Exclusión de variables booleanas y RainfallTmorrow porque no serán estandarizaradas
        exc_c = ["RainToday", "RainTomorrow", "RainfallTomorrow"]

        # Estandarización
        df_sub = X[[col for col in X.columns if col not in exc_c]]
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(df_sub)

        X_scaled = pd.DataFrame(X_scaled, columns=df_sub.columns)
        for col in exc_c:
            X_scaled[f"{col}"] = X[col]

        # Nuevo DataFrame estandarizado con los nombres de las columnas originales
        return X_scaled


class OutliersTreater(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        cols_with_ouliers = [
            "MinTemp",
            "MaxTemp",
            "Rainfall",
            "Evaporation",
            "Sunshine",
            "WindGustSpeed",
            "WindSpeed9am",
            "WindSpeed3pm",
            "Humidity9am",
            "Humidity3pm",
            "Pressure9am",
            "Pressure3pm",
            "Cloud9am",
            "Cloud3pm",
            "Temp9am",
            "Temp3pm",
        ]

        for col in cols_with_ouliers:
            IQR = X[col].quantile(0.75) - X[col].quantile(0.25)
            lower_bridge = X[col].quantile(0.25) - (IQR * 1.5)
            upper_bridge = X[col].quantile(0.75) + (IQR * 1.5)

            X.loc[X[col] >= round(upper_bridge, 2), col] = round(upper_bridge, 2)
            X.loc[X[col] <= round(lower_bridge, 2), col] = round(lower_bridge, 2)

        return X


class ColDropper(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.drop(["Date"], axis=1)


class LocDropper(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        costa_este = [
            " Adelaide",
            "Canberra",
            "Cobar",
            "Dartmoor",
            "Melbourne",
            "MelbourneAirport",
            "MountGambier",
            "Sydney",
            "SydneyAirport",
        ]
        X.loc[X["Location"].isin(costa_este), "Location"] = "costa_este"
        return X[X["Location"] == "costa_este"]


class CatFiller(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X["WindGustDir"] = X.groupby("Location")["WindGustDir"].transform(
            lambda x: x.fillna(x.mode()[0])
        )
        X["WindDir9am"] = X.groupby("Location")["WindDir9am"].transform(
            lambda x: x.fillna(x.mode()[0])
        )
        X["WindDir3pm"] = X.groupby("Location")["WindDir3pm"].transform(
            lambda x: x.fillna(x.mode()[0])
        )

        return X


class NumFiller(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        remanining_vnul_columns = X.columns[X.isna().any()].tolist()
        for col in remanining_vnul_columns:
            X[col] = X[col].fillna(X[col].mean())

        return X


# https://www.mdpi.com/2078-2489/13/4/163 Como las variables de la dirección de los vientos pueden tener hasta 16 direcciones diferentes, para convertirlos a variables numéricas, se tiene encuenta una distribución circular. Por eso, cada una de las variables se dividió en dos: Una con el seno y otra con el coseno del angulo
class CoordRecat(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        coord = {
            "N": 0,
            "NNE": 22.5,
            "NE": 45,
            "ENE": 67.5,
            "E": 90,
            "ESE": 112.5,
            "SE": 135,
            "SSE": 157.5,
            "S": 180,
            "SSW": 202.5,
            "SW": 225,
            "WSW": 247.5,
            "W": 270,
            "WNW": 292.5,
            "NW": 315,
            "NNW": 337.5,
        }

        # Aplicar la recategorización
        for col in ["WindGustDir", "WindDir9am", "WindDir3pm"]:
            X[col] = X[col].map(coord)
            X[f"{col}_rad"] = np.deg2rad(X[col])
            X[f"{col}_sin"] = np.sin(X[f"{col}_rad"]).round(5)
            X[f"{col}_cos"] = np.cos(X[f"{col}_rad"]).round(5)

        # Eliminar columnas originales y columnas radianes
        columns_to_drop = [
            f"{col}_rad" for col in ["WindGustDir", "WindDir9am", "WindDir3pm"]
        ] + ["WindGustDir", "WindDir9am", "WindDir3pm"]
        X = X.drop(columns=columns_to_drop, axis=1)

        return X


class LocEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        dummies = pd.get_dummies(X["Location"], dtype=int)
        X = pd.concat([X, dummies], axis=1)
        X.drop("Location", axis=1, inplace=True)

        return X


class ResetIndex(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.reset_index(drop=True)


class BoolYNDropperEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X.dropna(subset=["RainToday"], inplace=True)
        X["RainTomorrow"] = X["RainTomorrow"].map({"No": 0, "Yes": 1}).astype(float)
        X["RainToday"] = X["RainToday"].map({"No": 0, "Yes": 1}).astype(float)

        return X


class Standarizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Exclusión de variables booleanas y RainfallTmorrow porque no serán estandarizaradas
        exc_c = ["RainToday", "RainTomorrow"]

        # Estandarización
        df_sub = X[[col for col in X.columns if col not in exc_c]]
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(df_sub)

        X_scaled = pd.DataFrame(X_scaled, columns=df_sub.columns)
        for col in exc_c:
            X_scaled[f"{col}"] = X[col]

        # Nuevo DataFrame estandarizado con los nombres de las columnas originales
        return X_scaled


class OutliersTreater(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        cols_with_ouliers = [
            "MinTemp",
            "MaxTemp",
            "Rainfall",
            "Evaporation",
            "Sunshine",
            "WindGustSpeed",
            "WindSpeed9am",
            "WindSpeed3pm",
            "Humidity9am",
            "Humidity3pm",
            "Pressure9am",
            "Pressure3pm",
            "Cloud9am",
            "Cloud3pm",
            "Temp9am",
            "Temp3pm",
        ]

        for col in cols_with_ouliers:
            IQR = X[col].quantile(0.75) - X[col].quantile(0.25)
            lower_bridge = X[col].quantile(0.25) - (IQR * 1.5)
            upper_bridge = X[col].quantile(0.75) + (IQR * 1.5)

            X.loc[X[col] >= round(upper_bridge, 2), col] = round(upper_bridge, 2)
            X.loc[X[col] <= round(lower_bridge, 2), col] = round(lower_bridge, 2)

        return X


class RLValDropper(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X.dropna(subset=["RainTomorrow"], inplace=True)
        X.dropna(subset=["RainfallTomorrow"], inplace=True)
        return X

# DESCARTAR VARIABLES NO NUMERICAS Y ACOMODAR EL DATASET PARA ML OPS
# SOLAMENTE ML-OPS
cols = ['costa_este','WindGustDir_sin',	'WindGustDir_cos','WindDir9am_sin',	'WindDir9am_cos','WindDir3pm_sin','WindDir3pm_cos']
class DescartarNoUsarMlOPS(BaseEstimator, TransformerMixin):
    def fit(self, X,y=None):
        return self
    
    def transform(self,X):
        X = X.drop(cols, axis=1)
        return X


## Pipeline

# * Descartar Unnamed y Date porque son features que no vamos a utilizar: **ColDropper** (Ademas droppear RainfallTomorrow para prevenir la fuga de datos)
# * Descartar todas las location que no son necesarias: **LocDropper**
# * Dropear nulos y Label Encoding para las variables yes/no: **BoolYNDropperEncoder**
# * Imputar valores nulos en variables categoricas con la moda: **CatFiller**
# * Imputar valores nulos en variables numericas con la media:  **NumFiller**
# * One Hot Encoding para las location: **LocEncoder**
# * Encoding en sin y cos para WinDir: **CoordRecat**
# * Estandarizar valores: **Standarizer**

preprocessor = Pipeline(
    [
        ("drop_null_val_rl", RLValDropper()),
        ("drop_not_needed_features", ColDropper()),
        ("drop_nor_needed_locations", LocDropper()),
        ("yes_no_dropper_encoder", BoolYNDropperEncoder()),
        ("fill_null_cat", CatFiller()),
        ("fill_num_cat", NumFiller()),
        ("encode_loc", LocEncoder()),
        ("encode_wind_dir", CoordRecat()),
        ("reset_index", ResetIndex()),
        ("treat_outliers", OutliersTreater()),
        ("standariza_values", Standarizer()),
        ("Preparar_MLOPS", DescartarNoUsarMlOPS())
    ]
)


split

In [4]:
# Separación de variables explicativas y variables objetivo
X = df.drop(['RainTomorrow', ], axis=1).copy()
y = df[['RainTomorrow']].copy()

# Spliteo mi dataset en train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((116329, 23), (29083, 23), (116329, 1), (29083, 1))

In [5]:
# Creo un Dataframe de TRAIN
df_train = pd.DataFrame(X_train, columns=X.columns)
df_train['RainTomorrow'] = y['RainTomorrow']

In [6]:
# Creo un Dataframe de TEST
df_test = pd.DataFrame(X_test, columns=X.columns)
df_test['RainTomorrow'] = y['RainTomorrow']

In [7]:
# Preproceso mi df de test y mi df de train
df_train = preprocessor.fit_transform(df_train)
df_test = preprocessor.fit_transform(df_test)

In [8]:
df_train

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainfallTomorrow,RainToday,RainTomorrow
0,-0.095229,1.403916,-0.420677,0.129130,-0.329659,1.711644,0.740400,1.181787,-2.439476,-0.569634,-0.951037,-1.050733,-3.582069e-16,0.000000,1.424959,1.101909,0.079347,0.0,1.0
1,1.328003,2.268189,-0.625474,1.677469,-1.066479,1.110961,1.193486,-0.161767,-1.277913,-1.575068,-0.883153,-0.939793,5.415765e-01,1.414823,2.045610,2.347640,-0.317287,0.0,0.0
2,-0.685350,-1.219227,-0.625474,-0.926556,-0.565441,0.285021,0.966943,-0.049804,0.326149,0.888246,0.542412,0.752048,-6.683388e-01,0.983186,-1.217240,-1.187121,-0.256266,0.0,0.0
3,-0.164655,0.297041,-0.625474,-0.574661,0.908197,-1.892456,-0.958674,-1.281396,0.160212,-0.720449,1.384174,1.168075,-1.474949e+00,0.983186,0.165925,0.494615,-0.317287,0.0,0.0
4,1.171795,0.039275,1.934487,0.340267,-2.009607,0.585363,1.986387,0.845898,1.155836,0.687159,-0.475849,-0.301885,1.348187e+00,1.414823,0.485116,0.089753,0.140368,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19221,1.310647,0.236390,-0.215880,0.762541,-0.801224,2.500041,2.382838,-1.617284,-0.337601,1.943952,-1.371918,-1.286481,9.448817e-01,0.983186,1.318562,-0.299538,2.794764,0.0,1.0
19222,-0.112586,-0.582394,0.398510,-0.926556,0.407160,0.059765,0.740400,0.621973,-0.005726,-0.318275,0.664603,0.682711,5.415765e-01,0.983186,-0.366062,-0.548684,-0.317287,0.0,0.0
19223,0.043623,1.024849,-0.625474,0.129130,0.525051,-0.841260,-1.525031,-0.609619,-0.614163,-1.122623,-1.249727,-1.355819,-2.650337e-01,0.983186,0.396452,1.117481,0.140368,0.0,1.0
19224,0.564318,0.297041,-0.625474,1.184816,-0.771751,2.237242,2.382838,2.077490,-0.226976,0.335257,0.705334,0.544035,9.448817e-01,0.983186,0.378719,0.307756,0.964146,0.0,1.0


In [9]:
X_train_clasificacion = df_train.drop(['RainTomorrow', 'RainfallTomorrow'], axis=1).copy()
y_train_clasificacion = df_train['RainTomorrow'].copy()

X_test_clasificacion = df_test.drop(['RainTomorrow','RainfallTomorrow'], axis=1).copy()
y_test_clasificacion = df_test['RainTomorrow'].copy()

In [10]:
X_train_regresion = df_train.drop(['RainTomorrow','RainfallTomorrow'], axis=1).copy()
y_train_regresion = df_train['RainfallTomorrow'].copy()

X_test_regresion = df_test.drop(['RainTomorrow','RainfallTomorrow'], axis=1).copy()
y_test_regresion = df_test['RainfallTomorrow'].copy()

In [11]:
X_test_clasificacion

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,1.537261,1.019470,-0.632975,0.388774,1.188415,0.607450,-0.032009,1.426797,0.353163,0.769235,-0.181342,-0.586527,-1.475718,-1.628092e+00,1.453347,0.963325,0.0
1,0.836018,0.732462,-0.132392,0.744493,1.130197,0.017658,0.202022,0.167420,-1.543735,-1.097595,-0.921557,-1.122424,-1.475718,-2.065196e+00,0.578185,0.978937,0.0
2,-0.378330,1.321584,-0.632975,1.029068,1.508611,2.584719,-0.968133,2.514440,-1.766900,-1.803963,-0.307039,-0.755758,-1.475718,-7.538822e-01,0.875740,1.494115,0.0
3,1.383330,1.110104,-0.132392,-0.322663,1.421284,0.297290,-0.500071,0.625376,-0.260540,0.163776,0.461109,0.203214,0.139318,-1.190987e+00,1.435844,1.213108,0.0
4,2.324022,2.832153,-0.632975,2.451944,1.363067,-0.400570,-1.319180,-0.862979,-2.045856,-1.854418,-0.614298,-0.826270,-1.475718,-1.628092e+00,2.739834,2.805477,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4744,-0.788814,0.385031,-0.632975,0.531062,1.392176,-0.090410,-1.319180,-1.549911,0.074207,-0.845321,-0.684129,-0.628835,-1.071959,-1.628092e+00,0.000579,0.432536,0.0
4745,-0.446744,-0.536416,-0.632975,-0.251520,-0.761868,-0.943350,-1.787242,-0.634001,0.576327,0.062867,0.097985,0.033984,0.000000,-3.882270e-16,-0.524518,-0.519763,0.0
4746,-0.207295,1.170527,-0.632975,0.602206,1.741480,-0.710730,-0.734102,-0.862979,0.185789,-0.693956,0.544907,0.316035,0.000000,-3.882270e-16,0.245624,1.228720,0.0
4747,0.254499,-0.204091,1.869938,-0.536095,1.246632,1.150229,0.202022,0.854353,-1.711109,-0.946231,-0.628264,-0.346784,-1.475718,-1.628092e+00,0.123102,-0.176311,1.0


In [12]:
class RegLogistica:
    def __init__(self):
        self.parametros = {
            "C": 10,
            "class_weight": "balanced",
            "max_iter": 500,
            "solver": "newton-cg",
        }
        self.pipeline = None
        self.model = None
        self.y_pred_clasificacion = None

    def fit(self, X, y):
        if self.parametros:
            self.model = LogisticRegression(**self.parametros)
            self.model.fit(X_train_clasificacion, y_train_clasificacion)
        else:
            raise ValueError("Se necesita pasarle parametros al modelo")

        return self

    def predict(self, X):
        if self.model:
            y_pred_clasificacion = self.model.predict(X_test_clasificacion)
            return y_pred_clasificacion
        else:
            raise ValueError("hubo un error entrenando el modelo")

    def metrics(self, y_test_clasificacion):
        if y_test_clasificacion is None:
            raise ValueError("Se necesita el set de prueba")
        
        if self.y_pred_clasificacion is None:
            raise ValueError("Se necesita predecir")

        accuracy = accuracy_score(y_test_clasificacion, self.y_pred_clasificacion)
        precision = precision_score(y_test_clasificacion, self.y_pred_clasificacion)
        recall = recall_score(y_test_clasificacion, self.y_pred_clasificacion)
        f1 = f1_score(y_test_clasificacion, self.y_pred_clasificacion)

        metrics = {
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1,
        }

        return metrics


In [13]:
reg_logistica = RegLogistica()

In [14]:
# Entrenar el pipeline con los datos de entrenamiento
reg_logistica.fit(X_train_clasificacion, y_train_clasificacion)

# Hacer predicciones en los datos de prueba
y_pred_class = reg_logistica.predict(X_train_clasificacion)



In [15]:
y_test_clasificacion.shape, y_pred_class.shape

((4749,), (4749,))

In [16]:
from sklearn.metrics import recall_score, accuracy_score, precision_score
print(f"Recall: {recall_score(y_test_clasificacion, y_pred_class)}")
print(f"Precision: {precision_score(y_test_clasificacion, y_pred_class)}")
print(f"Accuracy: {accuracy_score(y_test_clasificacion, y_pred_class)}")

Recall: 0.7704918032786885
Precision: 0.5254658385093167
Accuracy: 0.7860602232048852
